<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<i>Material curado y desarrollado por Hernán D. Merlino</i><br> 
<i>AutoML - 2023 Q3</i></p> 
<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>AutoML Clustering</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:px 5px;">
Toda la información presentada en este trabajo fue extraída de:<br>    
<a href="http://epistasislab.github.io/tpot" target=\"_blank\">TPOT Web</a><br>
<a href="http://proceedings.mlr.press/v64/olson_tpot_2016.pdf" target=\"_blank\">TPOT: A Tree-based Pipeline Optimization Tool for Automating Machine Learning</a>    
</p>    

# TPOT
* TPOT es un aprendizaje automático automatizado de Python.
* Realiza una experimentación automática dentro de diferentes etapas del modelado:
  * Selección de features
  * Preprocesamiento de features
  * Construcción de features
  * Selección de modelos
  * Optimización de hiperparametros
* Las tareas de limpieza de datos y validación del modelo queda a cargo del usuario. 
* Utiliza internamente algoritmo genéticos para optimizar el rendimiento del modelo
* TPOT desarrolla un pipeline, evalúa su rendimiento y realiza modificaciones en el mismo buscando una mejora en el rendimiento
* Está construido sobre la librería Sklearn

image.png

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Uso</strong></p> 

In [ ]:
! pip install --quiet tpot

In [ ]:
# Importo las librerias
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import numpy as np

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Cargo el dataset como un dataframe
df = pd.read_csv('https://raw.githubusercontent.com/EpistasisLab/tpot/master/tutorials/Portuguese%20Bank%20Marketing/Data_FinalProject.csv')

In [ ]:
df.head()

age        job  marital  ... euribor3m nr.employed   y
0   56  housemaid  married  ...     4.857      5191.0  no
1   57   services  married  ...     4.857      5191.0  no
2   37   services  married  ...     4.857      5191.0  no
3   40     admin.  married  ...     4.857      5191.0  no
4   56   services  married  ...     4.857      5191.0  no

[5 rows x 21 columns]

In [ ]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [ ]:
# Convierto el target a numerico
df.y = df.y.map({'no':0, 'yes':1})

# TPot no tiene ningun preprocesamiento cuando la columna es de tipo string. Se tiene que realizar el preprocesamiento de forma manual
columnasObject = df.select_dtypes(include=np.object).columns
for columna in columnasObject:
  df = pd.concat([df.drop(columna, axis=1), pd.get_dummies(df[columna])], axis=1)

In [ ]:
df.head()

age  duration  campaign  pdays  ...  wed  failure  nonexistent  success
0   56       261         1    999  ...    0        0            1        0
1   57       149         1    999  ...    0        0            1        0
2   37       226         1    999  ...    0        0            1        0
3   40       151         1    999  ...    0        0            1        0
4   56       307         1    999  ...    0        0            1        0

[5 rows x 64 columns]

In [ ]:
# Separo el modleo entre train y test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['y'], axis=1), df.y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(30891, 63)
(10297, 63)
(30891,)
(10297,)


<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Configuracion Default</strong></p> 

In [ ]:
generations=2
population_size=5
pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1')

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/15 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5487713622402377

Generation 2 - Current best internal CV score: 0.5855008926466457

Best pipeline: GaussianNB(RandomForestClassifier(SGDClassifier(input_matrix, alpha=0.001, eta0=1.0, fit_intercept=True, l1_ratio=0.0, learning_rate=constant, loss=modified_huber, penalty=elasticnet, power_t=0.0), bootstrap=True, criterion=entropy, max_features=0.6000000000000001, min_samples_leaf=20, min_samples_split=10, n_estimators=100))


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=2,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory='auto', mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=42, scoring='f1', subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

0.5995542347696879


In [ ]:
pipeline_optimizer.export('tpot_export_default_pipeline.py')

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Configuracion Light</strong></p> 

In [ ]:
pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1',
    config_dict='TPOT light')

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/15 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5521113120653574

Generation 2 - Current best internal CV score: 0.5521113120653574

Best pipeline: KNeighborsClassifier(BernoulliNB(input_matrix, alpha=100.0, fit_prior=True), n_neighbors=21, p=1, weights=distance)


TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=2,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory='auto', mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=42, scoring='f1', subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

0.5605769230769231


In [ ]:
pipeline_optimizer.export('tpot_export_light_pipeline.py')

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Configuracion Custom</strong></p> 

In [ ]:
config_dict = {
     'sklearn.ensemble.RandomForestClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf':  range(1, 21),
        'bootstrap': [True, False]
    }
}

pipeline_optimizer = TPOTClassifier(
    generations=generations, 
    population_size=population_size, 
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1',
    config_dict=config_dict)

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/15 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5810115815350698

Generation 2 - Current best internal CV score: 0.5810115815350698

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'bootstrap': [True,
                                                                                      False],
                                                                        'criterion': ['gini',
                                                                                      'entropy'],
                                                                        'max_features': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
                                                                        'min_samples_leaf': range(1, 21),
                                                                        'min_samples_split': range(2, 21),
                                                                        'n_estimators': [100]}},
               crossover_rate=0.1, cv=5, disable_update_check=False,
               earl

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

0.594446973145198


In [ ]:
pipeline_optimizer.export('tpot_export_custom_pipeline.py')

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 